# To use files from google drive, need to run below

In [1]:
# Needed packages

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import timeit
import urllib.request  

import glob
import os
import shutil
import copy

In [2]:
# In the reading location, put the folder that contains the subfolders (each subfolder is a GRB) such that the subfolders contain separate .txt files for each of the GCNs
# reading_location     (GENERAL FOLDER WITH ALL GRBs)
#                '--------> 001122        (SUBFOLDER WITH A GRB)
#                                '-------> 001.txt       (TXT FILES EACH OF THEM IS A SEPARATE GCN FOR THE GRB 001122)
#                                '-------> 002.txt       (...)



reading_location = '/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified'
folderslist = [ f.path for f in os.scandir(reading_location) if f.is_dir() ] #going through all of the txt files inside the subfolders

In [3]:
start_time = timeit.default_timer()

positivelabel=['was corrected for galactic extinction','is corrected for galactic extinction',
               'were corrected for galactic extinction', 'are corrected for galactic extinction',
               'e(b-v)','E(B-V)','a(v)','a(b)','a(u)','a(r)','a(i)', 'E(B-V)', "A_v", "A_V", "Av", "A_I",
              'a(V)','a(B)','a(U)','a(R)','a(I)','we assume "typical" ctio extinction coefficients',
              'adopting the extinction curve of','the correct extinction values are',
              'corrected for a large galactic extinction','corrected for extinction of',
              'galactic extinction of','Corrected for the moderate Galactic foreground extinction',
              'After correction for foreground extinction','galactic extinction at this location',
              'In the extinction-corrected','assuming an SMC extinction law with',
              'Corrected for the moderate Galactic foreground extinction',
              'After correcting the interstellar extinction in our galaxy',
              'galactic extinction in U band is',
              'galactic extinction in B band is',
              'galactic extinction in V band is',
              'galactic extinction in R band is',
              'galactic extinction in I band is',
              'mag extinction in the observer frame V band assuming the mean Galactic dust+gas properties',
              'After correcting for a Galactic foreground extinction corresponding to a reddening of',
              'amount of SMC-like dust extinction',
              'After correcting for the foreground galactic extinction of',
              'Assuming an interstellar extinction obtained from',
              'and foreground extinction in this direction is low',
              'when taking into account the foreground Galactic extinction',
              'We also note the large amount of foreground extinction',
              'All the spectra have been corrected for the Galactic extinction',
              'After correcting for the foreground reddening of',
              'after correcting for galactic extinction',
              'Galactic foreground extinction was taken to be',
              'After correcting these magnitudes for the expected Galactic foreground extinction',
              'After correcting for the Galactic extinction corresponding to a reddening of',
              'and correcting for the (small) Galactic extinction',
              'correcting for Galactic extinction',
              'A standard Galactic extinction curve with',
              'although Galactic extinction in this direction is significant',
              'and Galactic extinction',
              'with a small amount of SMC extinction',
              'including SMC-like dust extinction',
              'that includes the foreground Galactic extinction',
              'broken power-law with a Milky Way extinction curve for the host extinction',
              'The resulting spectrum has been corrected for Galactic extinction',
              'corrected for a large galactic extinction',
              'at this position the expected E(B-V) reddening due to the Galactic foreground is',
              'passing through a galactic extinction of',
              'galactic extinction a(',
              'including the small galactic extinction',
              'extinction corrected spectrum',
              'the extinction coefficient was measured to be',  "after the correction for the Galactic extinction", "which have been corrected for",
               "when corrected for Galactic extinction", "a Galactic visual extinction along the line of sight", "Assuming no extinction", "adjusted by a Galactic extinction",
               "correcting the observed b-r for the galactic extinction", "this location suffers from significant Galactic extinction", "Recall that Galactic extinction in this direction",
               "notice an important Galactic extinction", "including the effects of Galactic extinction", "Due to a large Galactic extinction", "correcting for the small Galactic extinction",
               "corrected for galactic extinction", "after correction for Galactic extinction", "The Galactic extinction along the line of sight is low",
               "we assume a galactic extinction correction", "Galactic extinction included", "galactic extinction ar=02"
              ] 
negativelabel=["not corrected", "not been corrected", "No correction", "no correction", "without any correction", "without the galactic extinction correction"
               "No Galactic extinction correction",  "which when corrected for galactic extinction has", "taking into account the high foreground Galactic extinction in the line of sight",
               "very high excess N_H column density along the line of sight", "given the large galactic extinction towards this source", "uncorrected for Galactic extinction", 
               "did not apply Galactic extinction", "No Galactic extinction correction was applied", "without the galactic extinction correction", "on the galactic extinction curves", 
               "than can be accounted for by Galactic extinction", "not accounting for color terms or galactic foreground extinction", "most likely due to strong galactic dust extinction",
               "did not correct for Galactic foreground extinction", "with no evidence for extinction by dust", "uncorrected for the expected Galactic extinction",
               "uncorrected for the estimated Galactic extinction", "without correcting for the Galactic extinction", "no additional rest frame extinction"]
neg_flg=len(negativelabel)
description=copy.deepcopy(negativelabel)
description.extend(positivelabel)
used=np.zeros(len(description))
y_or_n_dict={}
for i, grbid in enumerate(folderslist):
    grb=grbid.replace(reading_location, "")
    print('GRB ' + grb)
    gcnslist=[g for g in glob.glob(grbid+"/"+"*.txt")]
    for j, gcnid in enumerate(gcnslist):
        gcn=gcnid.replace(reading_location, "")
        dict_tmp={gcn: "n"}
        with open(gcnid, errors="ignore") as f:
            lines=np.array(f.read().replace("(", "").replace(")", "").replace(".", "").replace(",", "").split())
            if ("extinction" in lines) and ("galactic" in lines or "Galactic" in lines):
                ind=np.where(lines=="extinction")[0][0]
                a=" ".join(lines[ind-10:ind+10])
                a_full=" ".join(lines)
                checksum= np.array([1 if des in a_full else 0 for des in description])
                #print(checksum, gcn)
                used += checksum
                if sum(checksum)==0:
                  print(a, gcn)
                  dict_tmp={gcn: "new"}
                elif sum(checksum) > 0 and checksum.nonzero()[0][0] >= neg_flg:
                  #print(checksum.nonzero()[0][0], "y")
                  dict_tmp[gcn]= "y"
                elif sum(checksum) > 0 and checksum.nonzero()[0][0] < neg_flg:
                  #print(checksum.nonzero()[0][0], "n")
                  _=1
                else:
                  print("Error")
            y_or_n_dict= {**y_or_n_dict, **dict_tmp}

elapsed = timeit.default_timer() - start_time
print(elapsed,' seconds')

0.0006537139997817576  seconds


In [4]:
# number of files with correction 
cnt=0
for s in y_or_n_dict.values():
  if s == "y":
    cnt += 1
cnt

0

In [5]:
# save dictionary as .txt file
with open('/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/code/Telescope_info_GCNs2.txt', 'w') as f:
  print("GRB GCN galextcorr\n", file=f)
  for k, v in y_or_n_dict.items():
    print(k.split(sep="/")[0], k.split(sep="/")[1].replace(".txt", ""), v, "\n", file=f)
    print(k, v)

In [9]:
###########################################################################################################################

In [6]:
cd "/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified"

/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified


In [7]:
# test if dictionary are in contradiction to written information

y_or_n_dict={}
with open("/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/code/if_galextcorr.txt") as f:
    for line in f.readlines()[2::2]:
        data = line.split(" ")
        key = str(data[0] + "/" + data[1] + ".txt")
        value=str(data[2])
        dict_tmp={key: value}
        y_or_n_dict= {**y_or_n_dict, **dict_tmp}
        
#print(y_or_n_dict)
import sys
checking_location= "/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/*.txt"
checkdir=glob.glob(checking_location, recursive=True)
checkdir=[i for i in checkdir if "(" not in i]
add_files=[]
cnt=0

#print(checkdir)
for filename in checkdir:
  add_flg=1
  grb_name=filename.split(sep="\\")[-1].split(sep="_")[0]
  print(filename)
  print("Checking " , grb_name, "...")
  with open(filename, "r") as f:
    lines= f.readlines()
  for line in lines:
    if len(line.split()) == 0:
      continue
    if line.split()[-2] == "y" and line.split()[-1].isdigit():
      add_flg=0
      check_tmp= grb_name +  "\\" + line.split()[-1] + ".txt"
      try:
        flg= (y_or_n_dict[check_tmp]=="y")
        cnt += 1-flg
        if flg==False:
          print("Contradiction", check_tmp)
      except:
        check_tmp=grb_name[:-1] + "\\" + line.split()[-1] + ".txt"
        flg= (y_or_n_dict[check_tmp]=="y")
        cnt += 1-flg
        if flg==False:
          print("Contradiction", check_tmp)
    elif line.split()[-2] == "n" and line.split()[-1].isdigit():
      add_flg=0
      check_tmp= grb_name +  "\\" + line.split()[-1] + ".txt"
      try:
        flg=(y_or_n_dict[check_tmp]== "n")
        cnt += 1-flg
        if flg==False:
          print("Contradiction", check_tmp)
      except:
        check_tmp=grb_name[:-1] + "\\" + line.split()[-1] + ".txt"
        flg= (y_or_n_dict[check_tmp]=="n")
        cnt += 1-flg
        if flg==False:
          print("Contradiction", check_tmp)
  if filename not in add_files and add_flg==1:
    add_files.append(filename)
if cnt ==0:
  print("No contradiction!")
print(checkdir)

/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel050502.txt
Checking  /media/darkwake/VIB2/GRB/data ...
/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel050803.txt
Checking  /media/darkwake/VIB2/GRB/data ...
/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel050814.txt
Checking  /media/darkwake/VIB2/GRB/data ...
/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel050915A.txt
Checking  /media/darkwake/VIB2/GRB/data ...
/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel050922B.txt
Checking  /media/darkwake/VIB2/GRB/data ...
/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel051006.txt
Checking  /media/darkwake/VIB2/GRB/data ...
/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel060111A.txt
Checking  /media/darkwake/VIB2/GRB/data ...
/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel060115.txt
Checking  /media/darkwake

In [8]:
print(len(checkdir))
len(add_files)

36


36

In [13]:
filename

'/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/tel050502.txt'

In [19]:
# Adding correction if not existed
for filename in add_files:
  print("Processing "+ filename.split(sep='\\')[-1].split(sep="_")[0] + "...")
  with open(filename, "r") as f:
    lines= f.readlines()
  
  filename = filename.replace('/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/', '/media/darkwake/VIB2/GRB/data_gathering/new_321_not_magenta/modified/GEC_added/')
  with open(filename, "w") as f:
    for i, line in enumerate(lines):
      if i==0:
        f.write("Time(s)\tMag\tMagErr\tFilter\tGalExtCorr\tSource\n")
        continue
      try:
        if len(line.split()) != 5:
          for j in range(len(line.split())-1):
            f.write(line.split()[j]+"\t")
          f.write(line.split()[-1]+"\n")
        else:
          if line.split()[-1].isdigit():
            check_tmp=filename.split(sep="/")[-1].split(sep="_")[0] + "/" + line.split()[-1] + ".txt"
            try:
              print(y_or_n_dict[check_tmp])
              for j in range(len(line.split())-1):
                f.write(line.split()[j]+"\t")
              f.write(y_or_n_dict[check_tmp]+"\t"+line.split()[-1]+"\n")
            except:
              try:
                check_tmp=filename.split(sep="/")[-1].split(sep="_")[0][:-1] + "/" + line.split()[-1] + ".txt"
                print(y_or_n_dict[check_tmp])
                for j in range(len(line.split())-1):
                  f.write(line.split()[j]+"\t")
                f.write(y_or_n_dict[check_tmp]+"\t"+line.split()[-1]+"\n")
              except:
                for j in range(len(line.split())-1):
                  f.write(line.split()[j]+"\t")
                f.write(line.split()[-1]+"\n")
          else:
            for j in range(len(line.split())-1):
              f.write(line.split()[j]+"\t")
            f.write(line.split()[-1]+"\n")
      except:
        continue

Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/GRB/data...
Processing /media/darkwake/VIB2/